# ibes_ltg

> Retrieve and process data on EPS Long-Term Growth (LTG) forecasts from WRDS IBES database.

***

The Forecast Period End Date (`fpedats`) is the ending month and year of the fiscal period to which the estimate applies (unless you're asking for a long-term-growth estimate, in which case the horizon is 3-5 years, so no explicity fiscal period is applicable). 

The Activation Date (`actdats`) is the date that the forecast/actual was recorded by Thomson Reuters. 

The Announce Date (`anndats`) is the date that the forecast/actual was reported. 

The Review Date (`revdats`) is most recent date that an estimate was confirmed as accurate. 

The Forecast Period Indicator (`fpi`) contains information about the horizon (how far into the future we are estimating). Key values: '0' for LTG, '1'-'5' for 1 to 5 years in the future, '6'-'9' for 1 to 4 quarters in the future. Farther horizons are available but they are extremely poorly populated.

Note that for Long Term Growth (LTG) estimates (`fpi='0'`), you must NOT select "Forecast Period End Date" as the Date Variable or the query will not return any estimates.

`TICKER` is the IBES ticker, which is not necessarily the same as the offical ticker of the firm.

***
It is possible for a contributing broker to provide multiple revisions to an estimate on the same day. In
this scenario, all estimates are available in the Detail history files and only the most current estimate is
included in the mean.

The Brokers (`estimator`) and Analysts (`analys`) are provided under numeric codes. 

***
**Estimate Revisions**

There are estimates which are dated “after” the announcement date. We
have no explanation other than the entry is in error.

Announcement of earnings will increment the FPI variable by 1 in all IBES
records for which review date (REVDATS)> report date (ANNDATS_ACT)

If at the time of the next review date the analyst at the same brokerage changes
her forecast for the same (TICKER, ANNDATS, FPEDATS, FPI, MEASURE,
USFIRM) combination, IBES will add a new observation. If the forecast
remains unchanged, IBES will not add new observations, but will adjust the
review date accordingly (REVDATS)

***

In [ ]:
#| default_exp wrds.ibes_ltg

In [ ]:
#|exports
from __future__ import annotations
from pathlib import Path
from typing import List
import os

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api
from finsets import RESOURCES

In [ ]:
#| export 
def raw_metadata(rawfile: str|Path=RESOURCES/'ibes_detu_epsus_variable_descriptions.csv', # location of the raw variable labels file
             ) -> pd.DataFrame:
    "Loads raw variable labels file, cleans it and returns it as a pd.DataFrame"

    df = pd.read_csv(rawfile)
    df['output_of'] = 'wrds.ibes_ltg.clean'

    df['Variable Label'] = df.apply(lambda row: row['Description'].replace(row['Variable Name'].strip()+' -- ', ''), axis=1)
    df['Variable Label'] = df.apply(lambda row: row['Variable Label'].replace( '(' + row['Variable Name'].strip() + ')', ''), axis=1)
    df['Variable Name'] = df['Variable Name'].str.strip().str.lower()
    df = df[['Variable Name', 'Variable Label', 'output_of', 'Type']].copy()
    df.columns = ['name','label','output_of','type']
    return df

In [ ]:
raw_metadata()

,name,label,output_of,type
0,oftic,Official Ticker,wrds.ibes_ltg.clean,Char
1,ticker,I/B/E/S Ticker,wrds.ibes_ltg.clean,Char
2,cusip,CUSIP/SEDOL,wrds.ibes_ltg.clean,Char
3,cname,Company Name,wrds.ibes_ltg.clean,Char
4,fpedats,"Forecast Period End Date, SAS Format",wrds.ibes_ltg.clean,Float
5,actdats,"Activation Date, SAS Format",wrds.ibes_ltg.clean,Date
6,acttims,"Activation Time, SAS Format",wrds.ibes_ltg.clean,Float
7,revdats,"Review Date, SAS Format",wrds.ibes_ltg.clean,Float
8,revtims,"Review Time, SAS Format",wrds.ibes_ltg.clean,Float
9,anndats,"Announce Date, SAS Format",wrds.ibes_ltg.clean,Date


In [ ]:
#| export
def raw_metadata_extra(wrds_username: str=None
             ) -> pd.DataFrame:
    "Collects metadata from WRDS `ibes.detu_epsus` table and merges it with `raw_metadata()`."

    if wrds_username is None:
        wrds_username = os.getenv('WRDS_USERNAME')
        if wrds_username is None: wrds_username = input("Enter your WRDS username: ") 

    try:
        db = wrds_api.Connection(wrds_username = wrds_username)
        funda = db.describe_table('ibes','detu_epsus')
        nr_rows = db.get_row_count('ibes','detu_epsus')
    finally:
        db.close()

    meta = funda[['name','type']].copy()
    meta['nr_rows'] = nr_rows
    meta['wrds_library'] = 'ibes'
    meta['wrds_table'] = 'detu_epsus'

    meta = meta.merge(raw_metadata()[['name','label']], how='left', on='name')
    
    meta['output_of'] = 'wrds.ibes_ltg.download'
    meta = pdm.order_columns(meta,these_first=['name','label','output_of'])
    for v in list(meta.columns):
        meta[v] = meta[v].astype('string')
    
    return meta

In [ ]:
#| eval: false
raw_metadata_extra()

Loading library list...
Done
Approximately 31823580 rows in ibes.detu_epsus.


,name,label,output_of,type,nr_rows,wrds_library,wrds_table
0,ticker,I/B/E/S Ticker,wrds.ibes_ltg.download,VARCHAR(6),31823580,ibes,detu_epsus
1,cusip,CUSIP/SEDOL,wrds.ibes_ltg.download,VARCHAR(8),31823580,ibes,detu_epsus
2,oftic,Official Ticker,wrds.ibes_ltg.download,VARCHAR(6),31823580,ibes,detu_epsus
3,cname,Company Name,wrds.ibes_ltg.download,VARCHAR(16),31823580,ibes,detu_epsus
4,actdats,"Activation Date, SAS Format",wrds.ibes_ltg.download,DATE,31823580,ibes,detu_epsus
5,estimator,Estimator,wrds.ibes_ltg.download,DOUBLE_PRECISION,31823580,ibes,detu_epsus
6,analys,Analyst Code,wrds.ibes_ltg.download,DOUBLE_PRECISION,31823580,ibes,detu_epsus
7,currfl,Canadian Currency Flag(Estimate Level),wrds.ibes_ltg.download,VARCHAR(1),31823580,ibes,detu_epsus
8,pdf,Primary/Diluted Flag(Estimate Level),wrds.ibes_ltg.download,VARCHAR(1),31823580,ibes,detu_epsus
9,fpi,<NA>,wrds.ibes_ltg.download,VARCHAR(1),31823580,ibes,detu_epsus


In [ ]:
#| export 
def default_raw_vars():
    return ['ticker', 'value', 'fpi', 'anndats', 'fpedats', 'revdats', 'actdats', 'estimator', 'analys', 'pdf']

In [ ]:
#| export
def download(vars: List[str]=None, # If None, downloads `default_raw_vars`; `permno`, `ticker`, and `anndats` added by default
             obs_limit: int=None, #Number of rows to download. If None, full dataset will be downloaded
             wrds_username: str=None, #If None, looks for WRDS_USERNAME with `os.getenv`, then prompts you if needed
             start_date: str=None, # Start date in MM/DD/YYYY format
             end_date: str=None, #End date in MM/DD/YYYY format; if None, defaults to current date
             permno_match_score: tuple=(1,), #accuracy of permno-ibes link. 1-6. 1 is best. use >1 with caution.
             ) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS `ibes.detu_epsus` library and adds PERMNO from CRSP"""

    if vars is None: vars = default_raw_vars()
    vars = ','.join(['a.ticker', 'a.anndats'] + 
                    [f'a.{x}' for x in vars if x not in ['ticker', 'anndats']])

    sql_string=f"""SELECT {vars}, b.permno
                        FROM ibes.detu_epsus AS a
                        LEFT JOIN wrdsapps_link_crsp_ibes.ibcrsphist AS b
                        ON a.ticker = b.ticker
                        WHERE a.anndats BETWEEN b.sdate AND b.edate
                """
    if permno_match_score is not None: sql_string += r" AND score IN %(permno_match_score)s"
    if start_date is not None: sql_string += r" AND anndats >= %(start_date)s"
    if end_date is not None: sql_string += r" AND anndats <= %(end_date)s"
    if obs_limit is not None: sql_string += r" LIMIT %(obs_limit)s"

    return wrds_api.download(sql_string, wrds_username=wrds_username, 
                             params={'permno_match_score': permno_match_score,
                                 'start_date':start_date, 'end_date':end_date, 'obs_limit':obs_limit})

In [ ]:
#| eval: false
ltg = download(permno_match_score=(1,2), start_date='01/01/2019', end_date='01/01/2022', obs_limit=1000)
ltg

Loading library list...
Done


,ticker,anndats,value,fpi,fpedats,revdats,actdats,estimator,analys,pdf,permno
0,0001,2019-01-04,-174.4,0,None,2019-01-04,2019-01-04,183.0,48368.0,D,14392.0
1,0001,2019-01-24,-173.7,0,None,2019-01-24,2019-01-24,183.0,48368.0,D,14392.0
2,0001,2019-03-17,-174.4,0,None,2019-03-17,2019-03-17,183.0,48368.0,D,14392.0
3,000R,2019-08-15,15.0,0,None,2019-12-20,2019-08-15,481.0,114029.0,D,14378.0
4,000Y,2019-09-18,-84.4,0,None,2019-09-24,2019-09-24,183.0,162718.0,D,14436.0
...,...,...,...,...,...,...,...,...,...,...,...
995,00UX,2019-12-17,-2.7,0,None,2020-01-20,2019-12-17,183.0,107332.0,D,15647.0
996,00UX,2020-02-11,7.5,0,None,2020-02-12,2020-02-12,183.0,107332.0,D,15647.0
997,00UX,2020-02-21,7.1,0,None,2020-03-16,2020-02-21,183.0,107332.0,D,15647.0
998,00UX,2020-03-25,5.4,0,None,2020-03-25,2020-03-25,183.0,107332.0,D,15647.0


In [ ]:
#| eval: false
annual = download(fpi=('1','2','3','4'), obs_limit=1000)
annual

Loading library list...
Done


,ticker,anndats,value,fpi,fpedats,revdats,actdats,estimator,analys,pdf,permno
0,0000,2014-03-09,0.73,1,2014-12-31,2014-03-11,2014-03-11,149.0,119962.0,D,14471.0
1,0000,2014-03-10,0.83,1,2014-12-31,2014-03-11,2014-03-11,228.0,80474.0,D,14471.0
2,0000,2014-03-10,0.80,1,2014-12-31,2014-03-11,2014-03-11,873.0,79092.0,D,14471.0
3,0000,2014-03-10,0.66,1,2014-12-31,2014-03-11,2014-03-11,952.0,50789.0,D,14471.0
4,0000,2014-03-17,0.86,1,2014-12-31,2014-04-07,2014-03-17,1267.0,71182.0,D,14471.0
...,...,...,...,...,...,...,...,...,...,...,...
995,0001,2015-07-31,-0.13,2,2016-12-31,2015-07-31,2015-07-31,118.0,78506.0,D,14392.0
996,0001,2015-07-31,0.30,2,2016-12-31,2015-07-31,2015-07-31,1267.0,107698.0,D,14392.0
997,0001,2015-07-23,-0.06,2,2016-12-31,2015-07-31,2015-07-31,3051.0,153433.0,D,14392.0
998,0001,2015-07-30,-0.16,2,2016-12-31,2015-08-12,2015-07-31,192.0,153101.0,D,14392.0


In [ ]:
#| eval: false
quarterly = download(fpi=('6','7','8','9'), obs_limit=1000)
quarterly

Loading library list...
Done


,ticker,anndats,value,fpi,fpedats,revdats,actdats,estimator,analys,pdf,permno
0,0000,2014-03-10,0.200,6,2014-01-31,2014-03-10,2014-03-10,1267.0,71182.0,D,14471.0
1,0000,2014-03-09,0.300,6,2014-03-31,2014-03-11,2014-03-11,149.0,119962.0,D,14471.0
2,0000,2014-03-10,0.350,6,2014-03-31,2014-03-11,2014-03-11,228.0,80474.0,D,14471.0
3,0000,2014-03-10,0.370,6,2014-03-31,2014-03-11,2014-03-11,873.0,79092.0,D,14471.0
4,0000,2014-03-10,0.230,6,2014-03-31,2014-03-11,2014-03-11,952.0,50789.0,D,14471.0
...,...,...,...,...,...,...,...,...,...,...,...
995,0001,2015-01-23,0.214,9,2015-09-30,2015-03-27,2015-01-23,100.0,104605.0,D,14392.0
996,0001,2015-01-26,0.280,9,2015-09-30,2015-01-26,2015-01-26,192.0,153101.0,D,14392.0
997,0001,2015-01-26,0.210,9,2015-09-30,2015-01-26,2015-01-26,3477.0,18677.0,D,14392.0
998,0001,2015-01-26,0.190,9,2015-09-30,2015-01-29,2015-01-26,42.0,137332.0,D,14392.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()